In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [4]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [6]:
gen = Generator(z_dim = 106)
load_model(gen, "U0A0 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_1 = gen(to_gen).detach().numpy()
y_1 = np.zeros(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A1 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_2 = gen(to_gen).detach().numpy()
y_2 = np.ones(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A2 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_3 = gen(to_gen).detach().numpy()
y_3 = np.ones(35) + 1

gen = Generator(z_dim = 106)
load_model(gen, "U1A0 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_4 = gen(to_gen).detach().numpy()
y_4 = np.ones(35) + 2

gen = Generator(z_dim = 106)
load_model(gen, "U1A1 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_5 = gen(to_gen).detach().numpy()
y_5 = np.ones(35) + 3

gen = Generator(z_dim = 106)
load_model(gen, "U1A2 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_6 = gen(to_gen).detach().numpy()
y_6 = np.ones(35) + 4

gen = Generator(z_dim = 106)
load_model(gen, "U2A0 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_7 = gen(to_gen).detach().numpy()
y_7 = np.ones(35) + 5

gen = Generator(z_dim = 106)
load_model(gen, "U2A1 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_8 = gen(to_gen).detach().numpy()
y_8 = np.ones(35) + 6

gen = Generator(z_dim = 106)
load_model(gen, "U2A2 Excluded Group 5_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_9 = gen(to_gen).detach().numpy()
y_9 = np.ones(35) + 7

X_test = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
y_test = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

In [7]:
activities = [1, 3, 4]
users = [23, 25, 27]
X_train, y_train = start_data(activities, users, "Activity", sub_features, act_features)

In [8]:
model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 6.619829177856445, Final Batch Loss: 2.197638750076294
Epoch 2, Loss: 6.648569345474243, Final Batch Loss: 2.2443320751190186
Epoch 3, Loss: 6.587665796279907, Final Batch Loss: 2.1790571212768555
Epoch 4, Loss: 6.585559129714966, Final Batch Loss: 2.187641143798828
Epoch 5, Loss: 6.624388217926025, Final Batch Loss: 2.2437262535095215
Epoch 6, Loss: 6.588857412338257, Final Batch Loss: 2.2103075981140137
Epoch 7, Loss: 6.566272974014282, Final Batch Loss: 2.1961586475372314
Epoch 8, Loss: 6.5700812339782715, Final Batch Loss: 2.2095401287078857
Epoch 9, Loss: 6.5267133712768555, Final Batch Loss: 2.1743485927581787
Epoch 10, Loss: 6.530782699584961, Final Batch Loss: 2.1883952617645264
Epoch 11, Loss: 6.453585624694824, Final Batch Loss: 2.1248645782470703
Epoch 12, Loss: 6.457779884338379, Final Batch Loss: 2.145509719848633
Epoch 13, Loss: 6.444777965545654, Final Batch Loss: 2.1576883792877197
Epoch 14, Loss: 6.367816925048828, Final Batch Loss: 2.0972516536712646
Ep

Epoch 128, Loss: 2.463742434978485, Final Batch Loss: 0.8756040334701538
Epoch 129, Loss: 2.3376263976097107, Final Batch Loss: 0.7665942311286926
Epoch 130, Loss: 2.184199869632721, Final Batch Loss: 0.6583002209663391
Epoch 131, Loss: 2.4381185173988342, Final Batch Loss: 0.9395009875297546
Epoch 132, Loss: 2.3675923943519592, Final Batch Loss: 0.7617121934890747
Epoch 133, Loss: 2.3294717669487, Final Batch Loss: 0.7748062610626221
Epoch 134, Loss: 2.489205300807953, Final Batch Loss: 0.9380477666854858
Epoch 135, Loss: 2.5087294578552246, Final Batch Loss: 0.9053663015365601
Epoch 136, Loss: 2.2570835947990417, Final Batch Loss: 0.7095527052879333
Epoch 137, Loss: 2.2963627576828003, Final Batch Loss: 0.7385416030883789
Epoch 138, Loss: 2.221068859100342, Final Batch Loss: 0.6743912100791931
Epoch 139, Loss: 2.5203492045402527, Final Batch Loss: 1.0324922800064087
Epoch 140, Loss: 2.2309271097183228, Final Batch Loss: 0.6892232894897461
Epoch 141, Loss: 2.241223931312561, Final Bat

Epoch 241, Loss: 1.975939393043518, Final Batch Loss: 0.6067597270011902
Epoch 242, Loss: 2.0347346663475037, Final Batch Loss: 0.6805775165557861
Epoch 243, Loss: 2.0059423446655273, Final Batch Loss: 0.6798161268234253
Epoch 244, Loss: 2.0292674899101257, Final Batch Loss: 0.650604784488678
Epoch 245, Loss: 2.026515543460846, Final Batch Loss: 0.6224198937416077
Epoch 246, Loss: 2.0488044023513794, Final Batch Loss: 0.6539190411567688
Epoch 247, Loss: 2.1211864948272705, Final Batch Loss: 0.7961387038230896
Epoch 248, Loss: 1.9999868273735046, Final Batch Loss: 0.7639214396476746
Epoch 249, Loss: 1.956190288066864, Final Batch Loss: 0.612602174282074
Epoch 250, Loss: 1.9613187313079834, Final Batch Loss: 0.7039548754692078
Epoch 251, Loss: 2.029394745826721, Final Batch Loss: 0.728575587272644
Epoch 252, Loss: 2.0284444093704224, Final Batch Loss: 0.7442853450775146
Epoch 253, Loss: 1.9892407059669495, Final Batch Loss: 0.6547983884811401
Epoch 254, Loss: 1.9979564547538757, Final Ba

Epoch 352, Loss: 1.8113875985145569, Final Batch Loss: 0.6981579661369324
Epoch 353, Loss: 1.722436934709549, Final Batch Loss: 0.6365589499473572
Epoch 354, Loss: 1.6417247951030731, Final Batch Loss: 0.46797290444374084
Epoch 355, Loss: 1.6319147944450378, Final Batch Loss: 0.5420902967453003
Epoch 356, Loss: 1.6197634935379028, Final Batch Loss: 0.5023446083068848
Epoch 357, Loss: 1.6617023348808289, Final Batch Loss: 0.6057009696960449
Epoch 358, Loss: 1.8543100357055664, Final Batch Loss: 0.6687543988227844
Epoch 359, Loss: 1.724384844303131, Final Batch Loss: 0.6125060319900513
Epoch 360, Loss: 1.8050233721733093, Final Batch Loss: 0.6299194693565369
Epoch 361, Loss: 1.8523091077804565, Final Batch Loss: 0.7397468090057373
Epoch 362, Loss: 1.7919533848762512, Final Batch Loss: 0.6553140878677368
Epoch 363, Loss: 1.6314950585365295, Final Batch Loss: 0.5407915711402893
Epoch 364, Loss: 1.8203009366989136, Final Batch Loss: 0.7171008586883545
Epoch 365, Loss: 1.5799390077590942, Fi

Epoch 462, Loss: 1.5068801641464233, Final Batch Loss: 0.5292260050773621
Epoch 463, Loss: 1.3406701385974884, Final Batch Loss: 0.4069027900695801
Epoch 464, Loss: 1.396308720111847, Final Batch Loss: 0.43953752517700195
Epoch 465, Loss: 1.4139597117900848, Final Batch Loss: 0.5039380192756653
Epoch 466, Loss: 1.5503033995628357, Final Batch Loss: 0.534761369228363
Epoch 467, Loss: 1.4420036971569061, Final Batch Loss: 0.3927944600582123
Epoch 468, Loss: 1.4506418704986572, Final Batch Loss: 0.4679446220397949
Epoch 469, Loss: 1.4801006019115448, Final Batch Loss: 0.5001624226570129
Epoch 470, Loss: 1.4079491198062897, Final Batch Loss: 0.40410134196281433
Epoch 471, Loss: 1.4893344044685364, Final Batch Loss: 0.5207774639129639
Epoch 472, Loss: 1.5837233364582062, Final Batch Loss: 0.603547215461731
Epoch 473, Loss: 1.427085280418396, Final Batch Loss: 0.4784703850746155
Epoch 474, Loss: 1.3491604328155518, Final Batch Loss: 0.33912497758865356
Epoch 475, Loss: 1.3990404307842255, Fi

...

Epoch 4507, Loss: 0.2647014334797859, Final Batch Loss: 0.08170058578252792
Epoch 4508, Loss: 0.36285897344350815, Final Batch Loss: 0.05826465040445328
Epoch 4509, Loss: 0.45149819552898407, Final Batch Loss: 0.20201624929904938
Epoch 4510, Loss: 0.3332829847931862, Final Batch Loss: 0.1197148934006691
Epoch 4511, Loss: 0.23209155350923538, Final Batch Loss: 0.03810727596282959
Epoch 4512, Loss: 0.3558754324913025, Final Batch Loss: 0.14850911498069763
Epoch 4513, Loss: 0.33621592819690704, Final Batch Loss: 0.1062137708067894
Epoch 4514, Loss: 0.3299129605293274, Final Batch Loss: 0.16022346913814545
Epoch 4515, Loss: 0.2708030864596367, Final Batch Loss: 0.0830310732126236
Epoch 4516, Loss: 0.3159126862883568, Final Batch Loss: 0.0685291439294815
Epoch 4517, Loss: 0.3249543011188507, Final Batch Loss: 0.13776159286499023
Epoch 4518, Loss: 0.3088781572878361, Final Batch Loss: 0.06095435097813606
Epoch 4519, Loss: 0.3647674694657326, Final Batch Loss: 0.1612696498632431
Epoch 4520, L

Epoch 4616, Loss: 0.48610279709100723, Final Batch Loss: 0.27485188841819763
Epoch 4617, Loss: 0.35569683462381363, Final Batch Loss: 0.1263762265443802
Epoch 4618, Loss: 0.2958154082298279, Final Batch Loss: 0.07248618453741074
Epoch 4619, Loss: 0.3197617083787918, Final Batch Loss: 0.10963337123394012
Epoch 4620, Loss: 0.34044988453388214, Final Batch Loss: 0.13751323521137238
Epoch 4621, Loss: 0.30349200963974, Final Batch Loss: 0.08092719316482544
Epoch 4622, Loss: 0.2952748015522957, Final Batch Loss: 0.049732863903045654
Epoch 4623, Loss: 0.3689730539917946, Final Batch Loss: 0.13483236730098724
Epoch 4624, Loss: 0.23615634813904762, Final Batch Loss: 0.06080830469727516
Epoch 4625, Loss: 0.3898979052901268, Final Batch Loss: 0.21168117225170135
Epoch 4626, Loss: 0.40315866470336914, Final Batch Loss: 0.22648313641548157
Epoch 4627, Loss: 0.2692171409726143, Final Batch Loss: 0.07874655723571777
Epoch 4628, Loss: 0.35683833807706833, Final Batch Loss: 0.13767053186893463
Epoch 46

Epoch 4725, Loss: 0.37612713873386383, Final Batch Loss: 0.1414613425731659
Epoch 4726, Loss: 0.2523181438446045, Final Batch Loss: 0.08298676460981369
Epoch 4727, Loss: 0.27896780520677567, Final Batch Loss: 0.10288671404123306
Epoch 4728, Loss: 0.430941641330719, Final Batch Loss: 0.10538917779922485
Epoch 4729, Loss: 0.36336204409599304, Final Batch Loss: 0.1306615024805069
Epoch 4730, Loss: 0.31934767961502075, Final Batch Loss: 0.07590359449386597
Epoch 4731, Loss: 0.27514181286096573, Final Batch Loss: 0.08419052511453629
Epoch 4732, Loss: 0.29424311220645905, Final Batch Loss: 0.1283578723669052
Epoch 4733, Loss: 0.34988246113061905, Final Batch Loss: 0.15470010042190552
Epoch 4734, Loss: 0.26409419253468513, Final Batch Loss: 0.04902709648013115
Epoch 4735, Loss: 0.2666783891618252, Final Batch Loss: 0.12211160361766815
Epoch 4736, Loss: 0.32799138873815536, Final Batch Loss: 0.09975694864988327
Epoch 4737, Loss: 0.2280326411128044, Final Batch Loss: 0.051645562052726746
Epoch 

Epoch 4834, Loss: 0.28309402614831924, Final Batch Loss: 0.07431855797767639
Epoch 4835, Loss: 0.39735599607229233, Final Batch Loss: 0.20732206106185913
Epoch 4836, Loss: 0.24724913015961647, Final Batch Loss: 0.05724116787314415
Epoch 4837, Loss: 0.261762086302042, Final Batch Loss: 0.04515950009226799
Epoch 4838, Loss: 0.2784951776266098, Final Batch Loss: 0.09416014701128006
Epoch 4839, Loss: 0.24793190509080887, Final Batch Loss: 0.07858175039291382
Epoch 4840, Loss: 0.2517433650791645, Final Batch Loss: 0.04464341327548027
Epoch 4841, Loss: 0.2564874403178692, Final Batch Loss: 0.051194656640291214
Epoch 4842, Loss: 0.23625003546476364, Final Batch Loss: 0.059510692954063416
Epoch 4843, Loss: 0.32010333240032196, Final Batch Loss: 0.14081905782222748
Epoch 4844, Loss: 0.35649145394563675, Final Batch Loss: 0.11529888212680817
Epoch 4845, Loss: 0.29508379846811295, Final Batch Loss: 0.08688142150640488
Epoch 4846, Loss: 0.20732558146119118, Final Batch Loss: 0.052969928830862045
E

Epoch 4945, Loss: 0.2766335979104042, Final Batch Loss: 0.09977003931999207
Epoch 4946, Loss: 0.3028443679213524, Final Batch Loss: 0.06658906489610672
Epoch 4947, Loss: 0.30929144844412804, Final Batch Loss: 0.11770842224359512
Epoch 4948, Loss: 0.2882499471306801, Final Batch Loss: 0.08860563486814499
Epoch 4949, Loss: 0.3176240548491478, Final Batch Loss: 0.08476704359054565
Epoch 4950, Loss: 0.3384006544947624, Final Batch Loss: 0.14760009944438934
Epoch 4951, Loss: 0.2296758070588112, Final Batch Loss: 0.06050165742635727
Epoch 4952, Loss: 0.34645141288638115, Final Batch Loss: 0.0494491271674633
Epoch 4953, Loss: 0.22244971990585327, Final Batch Loss: 0.043375447392463684
Epoch 4954, Loss: 0.39138974994421005, Final Batch Loss: 0.18590934574604034
Epoch 4955, Loss: 0.27449624985456467, Final Batch Loss: 0.06262490898370743
Epoch 4956, Loss: 0.3238535672426224, Final Batch Loss: 0.12807680666446686
Epoch 4957, Loss: 0.33317334949970245, Final Batch Loss: 0.10482857376337051
Epoch 

In [10]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[34  0  1  0  0  0  0  0  0]
 [34  1  0  0  0  0  0  0  0]
 [ 6  1 28  0  0  0  0  0  0]
 [ 0  0  0 35  0  0  0  0  0]
 [ 0  0  0  0 35  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 35]
 [ 0  0  0  1  0  0 34  0  0]
 [ 0  0  0  0  1  0  0 34  0]
 [ 0  0  4  0  0  7  0  0 24]]
              precision    recall  f1-score   support

         0.0    0.45946   0.97143   0.62385        35
         1.0    0.50000   0.02857   0.05405        35
         2.0    0.84848   0.80000   0.82353        35
         3.0    0.97222   1.00000   0.98592        35
         4.0    0.97222   1.00000   0.98592        35
         5.0    0.00000   0.00000   0.00000        35
         6.0    1.00000   0.97143   0.98551        35
         7.0    1.00000   0.97143   0.98551        35
         8.0    0.40678   0.68571   0.51064        35

    accuracy                        0.71429       315
   macro avg    0.68435   0.71429   0.66166       315
weighted avg    0.68435   0.71429   0.66166       315



In [12]:
# GET RID OF 2, 5, 8 for JUST DYNAMIC
(0.62385+0.05405+0.98592+0.98592+0.98551+0.98551)/6

0.7701266666666666